# Data analysis & prediction
David Gasquez & Diego Hueltes

## Tools
Useful tools for data analysis
- [Overview](#overview)
- [Jupyter](#jupyter)
- [Pandas](#pandas)
- [Seaborn](#seaborn)
- [bcolz](#bcolz)
- [Other tools](#other-tools)

## Data pipeline
Modeling workflow
- [Preprocesing](#preprocesing)
- [Analysis](#analysis)
- [Modeling](#modeling)
- [Model deployment](#deployment)
- [Model monitoring](#model-monitoring)

## Demo
Demo time!
- [Demo](#demo)

<a id='overview'></a>
# Overview

<img src="https://avatars3.githubusercontent.com/u/7388996" width="400">
<img src="http://www.numfocus.org/uploads/6/0/6/9/60696727/6893890_orig.png" width="600">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/Created_with_Matplotlib-logo.svg/2000px-Created_with_Matplotlib-logo.svg.png" width="300">
<img src="https://github.com/Blosc/bcolz/raw/master/doc/bcolz.png" width="300">
<img src="https://github.com/visualfabriq/bquery/raw/master/bquery.png" width="300">
<img src="https://avatars2.githubusercontent.com/u/365630?v=3&s=400" width="400">